# Pipeline e Bag of Words

Neste notebook serão feitas as transformações básicas no dataset. O objetivo com este notebook é gerar um csv que já contém todo o texto tokenizado e sem stop words. Além disso, sera gerado o resultado do modelo Bag of Words.
Na imagem abaixo, contém um diagrama de como essas etapas funcionam e no decorrer do notebook será entrado em mais detalhes sobre cada etapa específica.

## Importando as bibliotecas 

Nesta etapa são importadas e baixadas as bibliotecas e pacotes das mesmas, necessárias para as análises, gráficos, funções e para gerar o modelo Bag of Words.

In [10]:
pip install openpyxl



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
 pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import nltk 
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
nltk.download('punkt') 
nltk.download('stopwords') 
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Processamento dos dados 

### Importando a planilha 

Nesta váriavel é feito a importação da planilha que sera utilizada para gerar o bag of words

In [61]:
# Abrindo a base de dados 
df = pd.read_excel('../base/base.xlsx', index_col=0)

### Pré processamento dos dados 

Nesta etapa o foco é limpar da base de dados os dados que não serão utilizados.
Portanto, com essa função são removidos todos os comentários que tenham autoria do BTG Pactual e fotos de terceiros que marcavam a companhia.


In [62]:
# Esta função remove os dados que não serão utilizados para a análise
def clean_data(df):
    # Remove as aspas dos nomes das colunas
    df = df.rename(columns=lambda x: x.replace('\"', ''))
    # Remove comentários feito pelo btg
    dfWithoutAutor = df[df["autor"] != "btgpactual"]
    # Remove comentários nulos
    dfWithoutNull = dfWithoutAutor[dfWithoutAutor["autor"].notnull()]
    # Deixando apenas as interações do tipo comentário e resposta
    dfFinal = dfWithoutNull.loc[(dfWithoutNull['tipoInteracao'] == 'comentário') | (dfWithoutNull['tipoInteracao'] == 'resposta')]
    return dfFinal

#### Caso de teste

In [63]:
# Criando um data frame de teste para validar o pré processamento
data_test = {'id': [1, 2, 3, 4, 5],
             'dataPublicada': ['03/01/2022', '03/04/2022', '03/07/2022', '03/10/2022', '31/12/2022'],
             'autor': ['btgpactual', 'robert87', 'grace46', 'william25', 'john23'],
             'texto': ["Obrigado, 🚀🚀💙", "Bom dia, gostaria de saber como faço pra conseguir falar com um atendente humanizado no chat do btg banking?", 
                       "Bom dia, preciso urgente do meu informe de rendimentos do BTLG11, para declaração do meu IR. Já enviei vários e-mails e até o momento sem resposta. Telefone ninguém atende. Como faço?", "Pessoal, sempre muito bom mas se eu puder solicitar uma coisa seria legendas 😅", "Sempre é aproveitoso parar o q tá fazendo para ouvi-lo"],
             'sentimento': ['NEUTRAL', 'NEUTRAL', 'NEGATIVE', 'POSITIVE', 'POSITIVE'],
             'tipoInteracao': ["comentário", "comentário", "comentário", "marcação", "comentário"],
             'anomalia': [1, 0, 0, 1, 1],
             'probabilidadeAnomalia': [100, 20, 30, 100, 100],
             'linkPost': ['https://www.instagram.com/p/CapXhB5Lvas/#17935401550894179', 'https://www.instagram.com/p/CaqBiRpsYLm/', 'https://www.instagram.com/p/CaqBiRpsYLm/', 'https://www.instagram.com/p/CapXhB5Lvas/#17926096268158628', 'https://www.instagram.com/p/CaR1j2TuPpD/#17914683365221503'],
             'processado': [0,0,0,1,1],
             'contemHyperlink': [0,0,0,1,1]}
               
df_testing = pd.DataFrame(data_test)


In [64]:
# Testando a função
clean_data_test = df_testing
cleaning_test = clean_data(clean_data_test)

Os resultados foram colocados em células separadas para facilitar a visualização.

In [65]:
# Entrada do caso de teste
clean_data_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,03/01/2022,btgpactual,"Obrigado, 🚀🚀💙",NEUTRAL,comentário,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"Bom dia, preciso urgente do meu informe de ren...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
3,4,03/10/2022,william25,"Pessoal, sempre muito bom mas se eu puder soli...",POSITIVE,marcação,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17926...,1,1
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


In [66]:
# Resultado do caso de teste
cleaning_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"Bom dia, preciso urgente do meu informe de ren...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


##### Demonstração da função 

Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [68]:
cleaning_output = clean_data(df)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
id,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0
...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0


### Processamento dos dados

Nesta etapa as frases são tokenizadas (cada palavra da frase é dividida, sendo 1 palavra = 1 token) e removidas as stop words, palavras que não acrescentam significado semântico relevante (e.g: de, para, ou). Além disso, é gerado um arquivo .csv com uma nova coluna que contém os tratamento citados acima aplicado aos comentários (coluna 'texto'). 


In [69]:
def processing_comments(frase):
    #divide as frases em palavras
    palavras = frase.split() 
    #coloca todas as letras em minúsculas (lower) e remove as stop words
    filtered_words = [word for word in palavras if word.lower() not in stop_words
                        and not word.startswith('@') and word.isalpha()] 
    #tokeniza as palavras e organiza cada comentário em uma lista de palavras
    tokens = word_tokenize(" ".join(filtered_words)) 
    return tokens

#### Caso de teste

In [70]:
# Testando a função 
processing_test = cleaning_test['texto']
tokenize_test = processing_test.apply(processing_comments)

In [71]:
# Input da função
processing_test

1    Bom dia, gostaria de saber como faço pra conse...
2    Bom dia, preciso urgente do meu informe de ren...
4    Sempre é aproveitoso parar o q tá fazendo para...
Name: texto, dtype: object

In [72]:
# Resultado da função
tokenize_test

1    [Bom, gostaria, saber, faço, pra, conseguir, f...
2    [Bom, preciso, urgente, informe, rendimentos, ...
4         [Sempre, aproveitoso, parar, q, tá, fazendo]
Name: texto, dtype: object

##### Demonstração da função 

Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [73]:
processing_output = cleaning_output['texto'].apply(processing_comments)
processing_output

id
4                                      [Minuto, touro, ouro]
5                                                      [Sim]
6                [Queria, saber, Btg, banking, própria, btg]
11                                                        []
18                                                        []
                                 ...                        
3042413                              [Excelente, explicação]
3042713    [Porque, brg, tá, diminuído, limite, cartão, s...
3043812                      [Atendam, telefone, amor, Deus]
3044213    [grato, todo, esforço, rendeu, investimento, a...
3045012    [pq, morning, call, aparecendo, Spotify, dias,...
Name: texto, Length: 6356, dtype: object

### Aplicação do modelo 

Durante esta etapa é feita a implementação do modelo Bag of Words, que vetoriza as palavras e faz a contagem de vezes que cada uma aparece na coluna a ser utilizada. Assim, tendo como resultado um dataframe com as palavras que mais aparecem no texto e sua respectiva frequência.

In [75]:
def bow_dataframe(inputUser):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(inputUser)
    # Com essa variavel é possível ver as palavras e contar sua frequência
    wordCount = tokenizer.word_counts    
    dfCountBoW = pd.DataFrame(list(wordCount.items()))
    dfCountBoW.rename(columns={0: "Palavra", 1:"Frequência"}, inplace=True)
    final_df = dfCountBoW.sort_values(by=['Frequência'], ascending=False)
    return final_df

#### Caso de teste

In [76]:
# Testando a função 
bow_test = tokenize_test
bow_test_result = bow_dataframe(tokenize_test)

In [77]:
# Input da função
tokenize_test

1    [Bom, gostaria, saber, faço, pra, conseguir, f...
2    [Bom, preciso, urgente, informe, rendimentos, ...
4         [Sempre, aproveitoso, parar, q, tá, fazendo]
Name: texto, dtype: object

In [78]:
# Output da função
bow_test_result.head(5)

,Palavra,Frequência
0,bom,2
14,rendimentos,1
25,tá,1
24,q,1
23,parar,1


##### Demonstração da função 

Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [79]:
result_output = bow_dataframe(processing_output)
result_output.head(10)

,Palavra,Frequência
97,banco,475
6,btg,408
66,pra,386
590,limite,363
16,conta,319
299,cartão,310
188,melhor,228
149,agora,218
32,sempre,181
235,fazer,172


### Pipeline de processamento completo

Nesta etapa há todo o processamento da planilha em apenas uma função que retorna um csv completo das palavras já tokenizadas.

In [83]:
#Função para rodar nosso modelo de bag of words, essa função recebe um dataframe
def pipeline(base):
    #limpeza da base de dados
    base1 = clean_data(base)
    #tokenização da coluna de textos
    base1['Frases_sem_stop_words'] = base1['texto'].apply(processing_comments)
    return base1
    

#### Caso de teste

In [84]:
#Entrada
df_testing

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,03/01/2022,btgpactual,"Obrigado, 🚀🚀💙",NEUTRAL,comentário,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"Bom dia, preciso urgente do meu informe de ren...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
3,4,03/10/2022,william25,"Pessoal, sempre muito bom mas se eu puder soli...",POSITIVE,marcação,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17926...,1,1
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


In [85]:
# Resultado da função Pipeline
pipeline_output_test = pipeline(df_testing)
pipeline_output_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,Frases_sem_stop_words
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0,"[Bom, gostaria, saber, faço, pra, conseguir, f..."
2,3,03/07/2022,grace46,"Bom dia, preciso urgente do meu informe de ren...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0,"[Bom, preciso, urgente, informe, rendimentos, ..."
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1,"[Sempre, aproveitoso, parar, q, tá, fazendo]"


#### Baixar o resultado do modelo 

##### Demonstração da função 

Nesta etapa sera gerado o resultado da função e retornando todo o tratamento da base

In [86]:
result_pipeline = pipeline(df)

In [91]:
# Para salvar o resultado final basta apenas descomentar a linha a seguir
#result_pipeline.to_csv('..\\output\\Resultado-Processamento.csv',encoding='utf-8', index=False, header=True, )

### Resultado completo

Com todas etapas já feitas previamente, agora é a hora de rodar a limpeza dos dados e ter o resultado do bag of words.

In [88]:
# Limpando os dados originais
cleaned_df = pipeline(df)

In [90]:
# Resultado Bag of Words
result_final = bow_dataframe(cleaned_df['Frases_sem_stop_words'])
result_final.head(10)

,Palavra,Frequência
97,banco,475
6,btg,408
66,pra,386
590,limite,363
16,conta,319
299,cartão,310
188,melhor,228
149,agora,218
32,sempre,181
235,fazer,172
